In [1]:
# Dependencies
import requests
from pprint import pprint
import csv
import json
import datetime
import time
import pandas as pd

In [2]:
time_list = ['20190101+TO+20190531', '20120101+TO+20121231',
             '20180101+TO+20180531', '20180601+TO+20181231',
             '20170101+TO+20170531', '20170601+TO+20171231',
             '20160101+TO+20160531', '20160601+TO+20161231',
             '20150101+TO+20150531', '20150601+TO+20151231',
             '20140101+TO+20140531', '20140601+TO+20141231',
             '20130101+TO+20130531', '20130601+TO+20131231',
            ]
frames = []

for time_item in time_list:
    print (time_item)
    url = "https://api.fda.gov/food/enforcement.json?search=report_date:[" + time_item +"]&limit=99"
    print (url)
    respond = requests.get(url).json()
    df_temp = pd.DataFrame(respond["results"])
    df_clean = df_temp[['state', 'city', 'report_date', 'product_description', 'product_quantity', 'reason_for_recall', 'recalling_firm', 'recall_number']]
    frames.append(df_clean)
    
result = pd.concat(frames)

20190101+TO+20190531
https://api.fda.gov/food/enforcement.json?search=report_date:[20190101+TO+20190531]&limit=99
20120101+TO+20121231
https://api.fda.gov/food/enforcement.json?search=report_date:[20120101+TO+20121231]&limit=99
20180101+TO+20180531
https://api.fda.gov/food/enforcement.json?search=report_date:[20180101+TO+20180531]&limit=99
20180601+TO+20181231
https://api.fda.gov/food/enforcement.json?search=report_date:[20180601+TO+20181231]&limit=99
20170101+TO+20170531
https://api.fda.gov/food/enforcement.json?search=report_date:[20170101+TO+20170531]&limit=99
20170601+TO+20171231
https://api.fda.gov/food/enforcement.json?search=report_date:[20170601+TO+20171231]&limit=99
20160101+TO+20160531
https://api.fda.gov/food/enforcement.json?search=report_date:[20160101+TO+20160531]&limit=99
20160601+TO+20161231
https://api.fda.gov/food/enforcement.json?search=report_date:[20160601+TO+20161231]&limit=99
20150101+TO+20150531
https://api.fda.gov/food/enforcement.json?search=report_date:[20150

In [3]:
result.sort_values(by=['report_date'])
result = result.rename(columns={"recalling_firm":"brand"})

In [13]:
result.head()

,state,city,report_date,product_description,product_quantity,reason_for_recall,brand,recall_number
0,TX,Austin,20190306,Goat Cheese Salad with Mandarin Orange and Can...,16 units,Product potentially contaminated with Salmonella.,Whole Foods Market,F-1120-2019
1,TX,Austin,20190306,Focaccia Vegetable Pesto Sandwich packaged in ...,29 units,Product potentially contaminated with Salmonella.,Whole Foods Market,F-1119-2019
2,IL,Chicago,20190220,"RXBAR: Coffee Chocolate, 1.83oz bars, single b...","75,695,266 Individually wrapped bars",Chicago Bar Company LLC d/b/a RXBAR is volunta...,CHICAGO BAR COMPANY LLC RXBAR,F-0884-2019
3,IA,West des Moines,20190213,HyVee brand Chocolate-flavored Bettercreme che...,"2,608 32 oz and other similar sizes; 4,367 8 o...",Product contained an ingredient that tested po...,Hy-Vee Stores Inc,F-0831-2019
4,CA,Marina Del Rey,20190220,Thrive Market Non-GMO Creamy Almond Butter 16 ...,"152,216 all varieties",Potential Listeria monocytogenes contamination.,THRIVE MARKET,F-0906-2019


In [21]:
result['brand'] = result['brand'].str.lower()
result.head()

,state,city,report_date,product_description,product_quantity,reason_for_recall,brand,recall_number
0,TX,Austin,20190306,Goat Cheese Salad with Mandarin Orange and Can...,16 units,Product potentially contaminated with Salmonella.,whole foods market,F-1120-2019
1,TX,Austin,20190306,Focaccia Vegetable Pesto Sandwich packaged in ...,29 units,Product potentially contaminated with Salmonella.,whole foods market,F-1119-2019
2,IL,Chicago,20190220,"RXBAR: Coffee Chocolate, 1.83oz bars, single b...","75,695,266 Individually wrapped bars",Chicago Bar Company LLC d/b/a RXBAR is volunta...,chicago bar company llc rxbar,F-0884-2019
3,IA,West des Moines,20190213,HyVee brand Chocolate-flavored Bettercreme che...,"2,608 32 oz and other similar sizes; 4,367 8 o...",Product contained an ingredient that tested po...,hy-vee stores inc,F-0831-2019
4,CA,Marina Del Rey,20190220,Thrive Market Non-GMO Creamy Almond Butter 16 ...,"152,216 all varieties",Potential Listeria monocytogenes contamination.,thrive market,F-0906-2019


In [22]:
brands = list(result.brand.unique())
brands_df = pd.DataFrame(brands, columns=['brand'])
brands_df

,brand
0,whole foods market
1,chicago bar company llc rxbar
2,hy-vee stores inc
3,thrive market
4,oskri corp.
5,sysco charlotte
6,root 23 llc
7,"ardent mills, llc"
8,ventura foods llc
9,"diamond crystal brands, inc."


In [26]:
df = pd.read_csv('en.openfoodfacts.org.products.tsv', sep='\t', header=0)

In [27]:
df = df[['product_name', 'brands','ingredients_text', 'countries']]
df = df.loc[df['countries'] == 'US', :]
df = df[['product_name', 'brands','ingredients_text']]
df['brands'] = df['brands'].str.lower()

df = df.dropna()
df = df.reset_index(drop=True)
brands = list(df.brands.unique())
brands_df_new = pd.DataFrame(brands, columns=['brand'])


In [39]:
df = df.rename(columns={"brands":"brand"})

In [28]:
total_brand_list = pd.concat([brands_df, brands_df_new])

In [29]:
total_brand_list.nunique()
brands = list(total_brand_list.brand.unique())

In [30]:
id_brand_table = pd.DataFrame(brands, columns=['brand'])

In [31]:
id_brand_table.head()

,brand
0,whole foods market
1,chicago bar company llc rxbar
2,hy-vee stores inc
3,thrive market
4,oskri corp.


In [32]:
id_brand_table = id_brand_table.reset_index()
id_brand_table = id_brand_table.rename(columns={"index":"brand_id"})
id_brand_table.head()

,brand_id,brand
0,0,whole foods market
1,1,chicago bar company llc rxbar
2,2,hy-vee stores inc
3,3,thrive market
4,4,oskri corp.


In [33]:
id_brand_table.count()

brand_id    27942
brand       27942
dtype: int64

In [35]:
new_df = pd.merge(id_brand_table, result, on='brand', how='outer')
new_df = new_df[['brand_id', 'state', 'city', 'report_date', 'product_description', 'product_quantity', 'reason_for_recall', 'recall_number']]
new_df.head()


,brand_id,state,city,report_date,product_description,product_quantity,reason_for_recall,recall_number
0,0,TX,Austin,20190306,Goat Cheese Salad with Mandarin Orange and Can...,16 units,Product potentially contaminated with Salmonella.,F-1120-2019
1,0,TX,Austin,20190306,Focaccia Vegetable Pesto Sandwich packaged in ...,29 units,Product potentially contaminated with Salmonella.,F-1119-2019
2,0,TX,Austin,20190306,Serbian Ajvar Vegetable Club packaged in plast...,3 units,Product potentially contaminated with Salmonella.,F-1125-2019
3,0,TX,Austin,20190306,Spinach Artichoke Bleus Pizza packaged in plas...,2 units,Product potentially contaminated with Salmonella.,F-1126-2019
4,0,TX,Austin,20190306,Hot Bar Item; Mushroom Truffle Pasta Salad,1 case and 202.81 units,Product potentially contaminated with Salmonella.,F-1132-2019


In [40]:
new_df_1 = pd.merge(id_brand_table, df, on='brand', how='outer')
new_df_1 = new_df_1[['brand_id', 'product_name', 'ingredients_text']]
new_df_1.head()

,brand_id,product_name,ingredients_text
0,0,Pocket Pita Breads,"Whole wheat flour, water, honey, yeast and salt."
1,0,Cajun Remoulade,Ingredients: mayonnaise (expeller pressed cano...
2,0,Seafood Burger Sauce,"Mayo (expeller pressed canola oil, egg yolks, ..."
3,0,Chopped Kale Salad,"Kale, cindy's garlic tahini dressing hsh [filt..."
4,0,"Vegetable Bowl, Lemon Peppercorn & Parmesan Dip",Cindy's lemon peppercorn and parmesan dip (pre...
